<a href="https://colab.research.google.com/github/Tadesse-Destaw/Normalization/blob/main/NLP_Preprocessing_and_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amharic Text Preprocessing & Tokinization

In [52]:
!pip install flair
!pip install sweetviz
!pip install Unidecode
!pip install emoji
!pip install python-Levenshtein
!pip install import-ipynb

In [90]:
import sys
sys.path.append('../')

In [91]:
import importlib
from IPython.display import display
import sweetviz
import re
import random
import pandas as pd
import warnings
import unidecode
import numpy as np
import glob
import csv
import emoji
from typing import List
from flair.data import Token
from Levenshtein import distance 
import import_ipynb
from preprocessing.amharicSegmenter import AmharicSegmenter
#from Segmentation import remove_url
from normalizer import normalize
warnings.filterwarnings('ignore')

In [93]:
def find_url(string): 
    text = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',string)
    return "".join(text) # converting return value from list to string

In [94]:
def remove_url(data):
    return data.replace(find_url(data),"")

In [104]:
def remove_English_letters(news):
    for idx,cnt in enumerate(news):
        news[idx] =re.sub(r'[A-Za-z]','', cnt)  #\s+
    return news

def normalize(data):
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    return(normal.decode())

In [96]:
def remove_emoji(news):
    for idx,cnt in enumerate(news):
        news[idx] = re.sub(r'(:\S+)(@\w+)','', cnt)
    return news

In [97]:
def duplicated_values_data(data):
    dup=[]
    columns=data.columns
    for i in data.columns:
        dup.append(sum(data[i].duplicated()))
    return pd.concat([pd.Series(columns),pd.Series(dup)],axis=1,keys=['Columns','Duplicate count'])

In [105]:
remove = ['\xa087','\n','\xa0', '|', '/','፤','…','፣','«','(',')','"','“','”','»','-','.',
          '@','#','$','%','^','&','*','[',']','{','}',';','›','’','‘','‹','<','>','`','´','~','=','+','፦','፥','፧','፠']
May_news_DF = pd.DataFrame()
#May_news  = glob.glob("/srv/data/amharic/crawl/output/*.csv")
#May_news  = glob.glob("/srv/data/amharic/crawl/output/2020-1-28-0-0-3.csv")
May_news  = glob.glob("data/*.csv")
for news in May_news:
    data = pd.read_csv(news, names=["ID","URL","Date","Media","Content"] ,encoding="utf-8")
    #Removing duplicated sentences from Content
    data = data[data.Content.duplicated()==False].reset_index()
   
    for token in remove:
        data.Content = data.Content.apply(lambda x: x.replace(token,' ')) 
    #Removing the url from Content
    data.Content = data.Content.apply(lambda x: remove_url(x))
    data = data.drop(['index','ID','URL','Date','Media'], axis=1)
    #Merging the Dataframes
    May_news_DF = pd.concat([May_news_DF,data])

print("Before Removing Duplicated Sentences")
print(duplicated_values_data(May_news_DF))

May_news_DF = May_news_DF[May_news_DF.Content.duplicated()==False].reset_index()
May_news_DF = May_news_DF.drop(['index'], axis=1)
print("="*40,"\nAfter Removing Duplicated Sentences")
print(duplicated_values_data(May_news_DF))

#Splitting the sentences
#May_news_DF.Content = May_news_DF.Content.apply(lambda x: sentence_splitter(x))

#Remove Latin-alphabets from content 
May_news_DF.Content = May_news_DF.Content.apply(lambda x: remove_English_letters(x))

Before Removing Duplicated Sentences
   Columns  Duplicate count
0  Content              280
After Removing Duplicated Sentences
   Columns  Duplicate count
0  Content                0


TypeError: 'str' object does not support item assignment

In [34]:
#Number of Sentences
Sentence_Number = []
for x in May_news_DF.Content:
    Sentence_Number.append(len(x))

sum(Sentence_Number)

5873488

In [36]:
#May_news_DF.to_csv("MayNews.csv")
May_news_DF.Content.to_csv("MayNews_Content.csv", index=False)


In [39]:
#May_news_DF.Content[1]

# Generate Homophone words from the corpus




In [ ]:
hm1 =['ሀ','ሃ','ኀ','ኃ','ሐ','ሓ']
hm2 =['ሁ','ሑ','ኁ']
hm3 =['ህ','ሕ','ኅ']
hm4 =['ሰ','ሠ']
hm5 =['ስ','ሥ']
hm6 =['አ','ኣ','ዐ','ዓ']
hm7 =['ፀ','ጸ']
hm8 =['ፅ','ጽ']

In [40]:
df = pd.read_csv("MayNews_Content.csv")
df=pd.DataFrame(df)

In [ ]:
#Converting Dataframe to Lists
#News = df[df.Content.str.contains('|'.join(hm1))]
#News['Content'] = News.Content.apply(lambda x: str(x).split())
list_of_words = df['Content'].str.split(' ').to_list()

In [ ]:
print (len(list_of_words))

In [ ]:
#Converting Lists to Pandas Series
pd_series = pd.Series(list_of_words)
pd_series = pd_series.apply(pd.Series).stack().reset_index(drop = True) 
series_List = pd_series[pd_series.str.contains('|'.join(hm1))]            #words contains hml only


In [ ]:
series_List

In [ ]:
#Remove duplicated Valuees
test_words = set(series_List)
print (len(test_words))

In [44]:
target_word = 'ሀገር'
for hm in hm1:
  position=target_word.index(hm)
  if position >=0:
    break
target_word[:position] + '' + target_word[position+1:]

NameError: name 'hm1' is not defined

In [ ]:
words=set()
def match(target_word):
  for word in test_words:
    search_word =word.replace('|'.join(hm1),'')
    if distance(target_word, search_word) <=1:
      words.add(search_word)
match(target_word)       
print (words)

In [ ]:
import matplotlib.pyplot as plt  
series_List1 = pd_series[pd_series.str.contains('ሀ')] 
ሀ=len(series_List1)
series_List2 = pd_series[pd_series.str.contains('ሐ')] 
ሐ=len(series_List2)
series_List3 = pd_series[pd_series.str.contains('ኀ')] 
ኀ=len(series_List3)
series_List4 = pd_series[pd_series.str.contains('ሃ')] 
ሃ=len(series_List4)
series_List5 = pd_series[pd_series.str.contains('ሓ')] 
ሓ=len(series_List5)
series_List6 = pd_series[pd_series.str.contains('ኃ')] 
ኃ=len(series_List6)
data = {}
for variable in ["ሀ", "ሐ", "ኀ","ሃ","ሓ","ኃ"]:
    data[variable] = eval(variable)
import operator
sorted_d = sorted(data.items(), key=operator.itemgetter(1))
characters = list(data.keys()) 
no_words = list(data.values()) 
   
fig = plt.figure(figsize = (10, 6)) 
# creating the bar plot 
plt.bar(characters, no_words, color ='maroon') 
  
plt.xlabel("Homophone (ha) sound characters") 
plt.ylabel("Frequency of words") 
plt.title("Distribution of characters in words") 
plt.show()

## Homophone word Occurance Statistics

In [ ]:
homo_word = 'ሐገር'
result = list(filter(lambda x: homo_word in x, series_List))
#result = [i for i in series_List if homo_word in i] 
print (len(result))


## Print homophone words with context (the whole sentence)

In [42]:
f = open("all_sentences.txt", "r",encoding="utf-8")
content = f.read()
list_of_sentences = content.splitlines()
f.close()
#df = pd.read_csv("all_sentences.txt")
#News['Content'] = News.Content.apply(lambda x: str(x).split('([፡፡።?!])'))
#list_of_sentences = df['Content'].str.split('([፡፡።?!])').to_list()
sentences_series = pd.Series(list_of_sentences)
sentences_series = sentences_series.apply(pd.Series).stack().reset_index(drop = True) 
tagret_sentences = sentences_series[sentences_series.str.contains("ሀገር")]

In [43]:
tagret_sentences

1        በአስር ሺዎች የሚቆጠሩት ሰዎች ቀያቸውን ለቀው ወደ ጎረቤት ሀገር ወደ ሱ...
18       መጭው አዲስ ዓመት 2021 እና አፍሪቃ በአስቸጋሪ ሁኔታ የሚካሄዱ ምርጫዎ...
106      የቀድሞ የደርግ ባለስልጣናት ከ30 ዓመት በኋላ ተናገሩ በተለይ ለአዲስ አ...
578      በብልፅግና የሚመራው መንግስት ህዝብን ከህዝብ ለማጋጨት በሃይማኖታዊ እና ...
584      በብልፅግና የሚመራው መንግስት ህዝብን ከህዝብ ለማጋጨት በሃይማኖታዊ እና ...
                               ...                        
69209    እነዚህ ሁለት መሰረታዊ ጉዳዮች ማንም በቀላሉ ሊታዘበው የሚችልና እነሱም ...
69217    የቅናት ስሜት የተሰማኝ ለወገኖች ለሀገሬም ሆነ ለመንግስት እያበረከትኩት ...
69236    ማህበራዊ ኑሮአችን ግድ የሚሉንና ለተለያዩ የሀገር ግንባታ አላማዎች ከመን...
69243    ዳያስፖራዎች ካሉበት ሀገር እዚህ አገር ቤት ላሉት ዘመድ ወዳጆቻቸው የሚል...
69249    ከእኛ ሀገር ዳያስፖራዎች ይህን ያህል ዶላር በየወሩ የሚልክ ምን ያህሉ ይሆን?
Length: 713, dtype: object

In [ ]:
#homo_word = df[df['Content'].str.contains("ድህነት")]
#print("ድህነት", len(homo_word))
#ድህነት,ኅብር,ሕብር,ሳለ,ሣለ,ሰረገ,ሠረገ,ሰየመ,ሠየመ

## Homophone Characters Occurance

In [ ]:
#ሀ,ሃ,ኀ,ኃ,ሐ,ህ,ሕ,ኅ,አ,ዐ,ዓ,ሰ,ሠ,ሳ,ሣ,ስ,ሥ,ጸ,ፀ
hom_char = 0
for item in series_List:
    if 'ሃ' in item:
        hom_char = hom_char + 1
print(hom_char)

# Normalize based on frequency

In [ ]:
def normalizef(Content):
    for idx,cnt in enumerate(Content):
     # if len(homo_word)>len(homo_char_ha)
        Content[idx] = re.sub(r'[ሀኀኃሐሓኻ]','ሃ', cnt)
        Content[idx] = re.sub(r'[ጸ]','ፀ', cnt)
  return Content

In [ ]:
May_news_DF.Content = May_news_DF.Content.apply(lambda x: normalizef(x))

# Normalize based on litrature



In [ ]:
May_news_DF.Content = May_news_DF.Content.apply(lambda x: normalize(x))
#May_news_DF.Content[0:10]

# Amharic Text Segmentation (sentence and tokens)

In [41]:
sent_punct = []
word_punct = []
amseg = AmharicSegmenter(sent_punct,word_punct)
#text=pd.DataFrame(df)
text = df.to_string(index = False) 
#text  =  "በዚህም መሠረት የአማራ:: ዴሞክራሲያዊ ኃይል ንቅናቄ የመረጠው: ምልክት ከዚህ በፊት በነበረ፡፡ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው፡፡ የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የጋሞ ዴሞክራሲያዊ ፓርቲ የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የኢትዮጵያ ዜጎች ለማኅበራዊ ፍትሕ (ኢዜማ) የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የምዕራብ ሶማሌ ዴሞክራሲያዊ ፓርቲ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ፣ እንዲሁም የአርጎባ ብሔረሰብ ዴሞክራሲያዊ ንቅናቄ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ ነው እንዲቀይሩ በምርጫ ቦርድ ማሳሰቢያ የተሰጣቸው፡፡"
with open ("all_sentences.txt","w", encoding="utf-8") as all_sentences:
  for s in amseg.tokenize_sentence(text):
      all_sentences.write(s+'\n')

In [ ]:
text  =  "በዚህም መሠረት የአማራ:: ዴሞክራሲያዊ ኃይል ንቅናቄ የመረጠው: ምልክት ከዚህ በፊት በነበረ፡፡ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው፡፡ የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የጋሞ ዴሞክራሲያዊ ፓርቲ የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የኢትዮጵያ ዜጎች ለማኅበራዊ ፍትሕ (ኢዜማ) የመረጠው ምልክት ከዚህ በፊት በነበረ ምርጫ ሌላ ፓርቲ ሲጠቀምበት የነበረ በመሆኑና ፓርቲው የቀድሞ ምልክቱን ለመጠቀም በመጠየቁ፣ የምዕራብ ሶማሌ ዴሞክራሲያዊ ፓርቲ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ፣ እንዲሁም የአርጎባ ብሔረሰብ ዴሞክራሲያዊ ንቅናቄ ከሌላ ፓርቲ ምልክት ጋር ምልክቱ የተቀራረበ በመሆኑ ነው እንዲቀይሩ በምርጫ ቦርድ ማሳሰቢያ የተሰጣቸው፡፡"
#with open ("all_tokens.txt","w", encoding="utf-8") as all_tokens:
amseg = AmharicSegmenter(sent_punct,word_punct)
amseg.amharic_tokenizer(text)
  